In [28]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, IntegerLookup, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
import os
import matplotlib.pyplot as plt
import numpy as np
import cv2

In [29]:
# train_dir = 'testdata/images 2/train'
# validation_dir = 'testdata/images 2/validation'

train_dir = 'emotiondataset/images/images/train'
validation_dir = 'emotiondataset/images/images/validation'

In [30]:
train_datagen = ImageDataGenerator()

validation_datagen = ImageDataGenerator()


# Create data generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(48, 48),
    batch_size=32,
    color_mode='grayscale',
    class_mode='categorical',
    shuffle=True  # Add shuffling here
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(48, 48),
    batch_size=32,
    color_mode='grayscale',
    class_mode='categorical',
    shuffle=False  # Add shuffling here
)

Found 28386 images belonging to 7 classes.
Found 7066 images belonging to 7 classes.


In [31]:
model = Sequential([
    Conv2D(64, (3, 3), activation='relu',padding='same', input_shape=(48, 48, 1)),
    MaxPooling2D(2, 2),
    BatchNormalization(),
    Dropout(0.2),
    Conv2D(128, (5, 5), activation='relu',padding='same'),
    MaxPooling2D(2, 2),
    BatchNormalization(),
    Dropout(0.2),
    Conv2D(128, (3, 3), activation='relu',padding='same'),
    MaxPooling2D(2, 2),
    BatchNormalization(),
    Dropout(0.2),
    BatchNormalization(),
    Dropout(0.5),
    Flatten(),
    Dense(1024, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),
    Dense(1024, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),
    Dense(7, activation='softmax')  # Assuming 7 emotions
    
])

In [32]:
model.compile(optimizer='adam',  # Adjusted initial learning rate
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 48, 48, 64)        640       
                                                                 
 max_pooling2d_9 (MaxPoolin  (None, 24, 24, 64)        0         
 g2D)                                                            
                                                                 
 batch_normalization_20 (Ba  (None, 24, 24, 64)        256       
 tchNormalization)                                               
                                                                 
 dropout_20 (Dropout)        (None, 24, 24, 64)        0         
                                                                 
 conv2d_10 (Conv2D)          (None, 24, 24, 128)       204928    
                                                                 
 max_pooling2d_10 (MaxPooli  (None, 12, 12, 128)      

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=100,  # Increased epochs
    validation_data=validation_generator,
    validation_steps=len(validation_generator)
)

Epoch 1/100


2024-05-05 23:20:09.467801: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


888/888 [==============================] - ETA: 0s - loss: 1.8699 - accuracy: 0.3056

2024-05-05 23:20:34.476095: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


888/888 [==============================] - 28s 30ms/step - loss: 1.8699 - accuracy: 0.3056 - val_loss: 2.3645 - val_accuracy: 0.3344
Epoch 2/100
888/888 [==============================] - 28s 32ms/step - loss: 1.5684 - accuracy: 0.3869 - val_loss: 2.2450 - val_accuracy: 0.4311
Epoch 3/100
888/888 [==============================] - 32s 36ms/step - loss: 1.4405 - accuracy: 0.4342 - val_loss: 2.4538 - val_accuracy: 0.4144
Epoch 4/100
888/888 [==============================] - 30s 33ms/step - loss: 1.3700 - accuracy: 0.4668 - val_loss: 2.3858 - val_accuracy: 0.4482
Epoch 5/100
888/888 [==============================] - 30s 34ms/step - loss: 1.3251 - accuracy: 0.4864 - val_loss: 2.4389 - val_accuracy: 0.4357
Epoch 6/100
888/888 [==============================] - 28s 32ms/step - loss: 1.2922 - accuracy: 0.5013 - val_loss: 2.6026 - val_accuracy: 0.3984
Epoch 7/100
888/888 [==============================] - 30s 34ms/step - loss: 1.2591 - accuracy: 0.5148 - val_loss: 2.4668 - val_accuracy: 0.47

In [ ]:
validation_loss, validation_accuracy = model.evaluate(validation_generator)
print(f'Validation Accuracy: {validation_accuracy * 100:.2f}%')

In [ ]:

plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()


In [ ]:
model.save('emotion_detection_model.h5')

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns


# Predict classes on validation data
y_pred = model.predict(validation_generator)
y_pred_classes = np.argmax(y_pred, axis=1)
true_classes = validation_generator.classes

# Generate confusion matrix
conf_mat = confusion_matrix(true_classes, y_pred_classes)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_mat, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

# Calculate emotion-specific metrics
emotion_labels = validation_generator.class_indices.keys()
class_report = classification_report(true_classes, y_pred_classes, target_names=emotion_labels)
print(class_report)


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns


# Generate confusion matrix and emotion-specific metrics
y_pred = model.predict(validation_generator)
y_pred_classes = np.argmax(y_pred, axis=1)
true_classes = validation_generator.classes
conf_mat = confusion_matrix(true_classes, y_pred_classes)
emotion_labels = validation_generator.class_indices.keys()
class_report = classification_report(true_classes, y_pred_classes, target_names=emotion_labels)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_mat, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

# Print emotion-specific metrics
print(class_report)
